# 06 – Model Evaluation
This notebook evaluates classification models (Logistic Regression and Random Forest) on the synthetic dataset.
It outputs:
- `results/model_performance_summary.csv`
- `results/classification_reports.txt`
- `results/roc_logreg.png`, `results/roc_random_forest.png`
- `results/cm_logreg.png`, `results/cm_random_forest.png`


In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (classification_report, confusion_matrix, roc_curve,
                             roc_auc_score, accuracy_score, precision_score, recall_score, f1_score)
import matplotlib.pyplot as plt
import numpy as np
import os

# Resolve data path
candidates = [
    '../data/cleaned_reports.csv',
    '../data/sample_reports_100.csv',
    '/mnt/data/cleaned_reports.csv',
    '/mnt/data/sample_reports_100.csv',
    '../data/sample_reports.csv',
    '/mnt/data/sample_reports.csv'
]
data_path = next((p for p in candidates if Path(p).exists()), None)
assert data_path is not None, f'Could not find dataset in: {candidates}'
print('Using data:', data_path)

df = pd.read_csv(data_path, parse_dates=['timestamp'])
df['target'] = (df['status'].str.upper() == 'FAILED').astype(int)

features = ['system_name', 'error_code', 'response_time_ms', 'cpu_usage', 'memory_usage', 'cost_usd']
X = df[features]
y = df['target']

cat_features = ['system_name', 'error_code']
num_features = ['response_time_ms', 'cpu_usage', 'memory_usage', 'cost_usd']

preprocess = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
    ('num', 'passthrough', num_features)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# Define models
models = {
    'logreg': LogisticRegression(max_iter=300),
    'random_forest': RandomForestClassifier(n_estimators=300, random_state=42)
}

performance_rows = []
reports_text = []

# Ensure results dir
res_dir = Path('../results')
res_dir.mkdir(parents=True, exist_ok=True)

for name, estimator in models.items():
    pipe = Pipeline(steps=[('prep', preprocess), ('clf', estimator)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    y_proba = getattr(pipe, 'predict_proba', lambda X: None)(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = roc_auc_score(y_test, y_proba[:,1]) if y_proba is not None else np.nan
    
    performance_rows.append({
        'model': name,
        'accuracy': round(acc, 4),
        'precision': round(prec, 4),
        'recall': round(rec, 4),
        'f1': round(f1, 4),
        'roc_auc': round(auc, 4) if not np.isnan(auc) else ''
    })
    
    # Classification report
    cr = classification_report(y_test, y_pred)
    reports_text.append(f'=== {name.upper()} ===\n{cr}\n')
    
    # Confusion matrix plot
    cm = confusion_matrix(y_test, y_pred)
    plt.figure()
    plt.imshow(cm, interpolation='nearest')
    plt.title(f'Confusion Matrix – {name}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    # Annotate
    for (i, j), v in np.ndenumerate(cm):
        plt.text(j, i, str(v), ha='center', va='center')
    plt.tight_layout()
    plt.savefig(res_dir / f'cm_{name}.png')
    plt.close()
    
    # ROC curve plot (one figure per model)
    if y_proba is not None:
        fpr, tpr, _ = roc_curve(y_test, y_proba[:,1])
        plt.figure()
        plt.plot(fpr, tpr, label=f'{name}')
        plt.plot([0,1], [0,1], linestyle='--')
        plt.title(f'ROC Curve – {name}')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.tight_layout()
        plt.savefig(res_dir / f'roc_{name}.png')
        plt.close()

# Save performance CSV and reports
perf_df = pd.DataFrame(performance_rows)
perf_df.to_csv(res_dir / 'model_performance_summary.csv', index=False)
with open(res_dir / 'classification_reports.txt', 'w') as f:
    f.write('\n'.join(reports_text))

perf_df
